In [4]:
import pandas as pd
from resources import *
import json

df = pd.read_csv('../short.csv')
extracted_skills_column = []

for index, row in df.iterrows():
    text = row['cleaned_description']
    resume_ngrams = extract_ngrams_from_text(text)  # Generate n-grams
    resume_skills = match_skills(resume_ngrams, text)  # Match skills
    education_skills = resume_skills["Education Certification (EC)"]
    predictions = predict_entities(text)
    outside_words = predictions["Outside (O)"]  # Extract "Outside" words/phrases from Model 2
    outside_text = preprocess_text(" ".join(outside_words))
    outside_ngrams = extract_ngrams_from_text(outside_text)  # Generate n-grams
    skills_from_outside = match_skills(outside_ngrams, outside_text)  # Match skills

    for category, skills in predictions.items():
        if category != "Outside (O)" and category != "Education Certification (EC)":
            for word in skills:
                skills_from_outside[category].add(preprocess_text(word))
    
    skills_from_outside["Education Certification (EC)"] = education_skills
    outside_skills_dict = {key:list(itemList) for key,itemList in skills_from_outside.items()}

    extracted_skills = json.dumps(outside_skills_dict)  # Convert dict to string for storage
    extracted_skills_column.append(extracted_skills)

df["extracted_skills"] = pd.Series(extracted_skills_column)

df.to_csv("extracted_job_skills.csv")